In [1]:
import sys
import random
random.seed(42)
import numpy as np
np.random.seed(42)
from keras import backend as K
from tensorflow import set_random_seed
set_random_seed(42)


from keras.optimizers import Adam, SGD
from keras.callbacks import Callback, CSVLogger, ModelCheckpoint
from tqdm import tqdm
from sklearn.utils import class_weight

Using TensorFlow backend.


In [0]:
from imblearn.keras import balanced_batch_generator


**VGG**

In [0]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np


def preprocess_input_vgg(x):
      """Wrapper around keras.applications.vgg16.preprocess_input()
      to make it compatible for use with keras.preprocessing.image.ImageDataGenerator's
      `preprocessing_function` argument.

      Parameters
      ----------
      x : a numpy 3darray (a single image to be preprocessed)

      Note we cannot pass keras.applications.vgg16.preprocess_input()
      directly to to keras.preprocessing.image.ImageDataGenerator's
      `preprocessing_function` argument because the former expects a
      4D tensor whereas the latter expects a 3D tensor. Hence the
      existence of this wrapper.

      Returns a numpy 3darray (the preprocessed image).

      """
      from keras.applications.vgg16 import preprocess_input
      X = np.expand_dims(x, axis=0)
      X = preprocess_input(X)
      return X[0]


def get_VGG_data_generator(file_num, file_name, fraction):
  print('VGG DATA GENERATOR')


  import os
  import zipfile
#   local_zip = 'gdrive/My Drive/Colab Notebooks/UC_Merced/' + file_name
#   local_zip = 'gdrive/My Drive/Colab Notebooks/Sint_Maarten/' + 'shape/' + str(fraction) + '/'+ file_name
  local_zip = 'gdrive/My Drive/Colab Notebooks/Sint_Maarten/' + 'material/' + str(fraction) + '/'+ file_name


  print(local_zip)
  zip_ref = zipfile.ZipFile(local_zip, 'r')
  zip_ref.extractall('/tmp')
  zip_ref.close()

  base_dir = '/tmp/' + file_name.split(".")[0]
  # base_dir = '/tmp/UC_Merced_filtered_45'

  print(base_dir)

  train_dir = os.path.join(base_dir, 'train')
  validation_dir = os.path.join(base_dir, 'validation')
  test_dir = os.path.join(base_dir, 'test')


  # Adding rescale, rotation_range, width_shift_range, height_shift_range,
  # shear_range, zoom_range, and horizontal flip to our ImageDataGenerator
  train_datagen = ImageDataGenerator(
      preprocessing_function=preprocess_input_vgg,
      rotation_range=180,
      horizontal_flip=True,
      vertical_flip=True
  )

  # Note that the validation and test data should not be augmented!
  val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg)
  test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input_vgg)

  # Flow training images in batches of 16 using train_datagen generator
  train_generator = train_datagen.flow_from_directory(
          train_dir,  # This is the source directory for training images
          target_size=(224, 224),  # All images will be resized to 150x150
          batch_size=16,
          # Since we use binary_crossentropy loss, we need binary labels
          class_mode='sparse', 
          shuffle=True)

  # Flow validation images in batches of 16 using test_datagen generator
  validation_generator = val_datagen.flow_from_directory(
          validation_dir,
          target_size=(224, 224),
          batch_size=16,
          class_mode='sparse')

  # Flow test images in batches of 16 using test_datagen generator
  test_generator = test_datagen.flow_from_directory(
          test_dir,
          target_size=(224, 224),
          batch_size=100,
          class_mode='sparse')
  
  return train_generator, validation_generator, test_generator



def get_VGG(n_classes):
  print('VGG MODEL')
  
  from keras.applications.vgg16 import  VGG16
  from keras.models import Model
  from keras.layers import Dense, GlobalAveragePooling2D, Input, Flatten
  from keras.layers import Dropout, BatchNormalization

  
  base_model  = VGG16(weights='imagenet', include_top=False, input_shape=(224,224, 3))
#   base_model = Model(inputs=base_model.input, outputs=base_model.get_layer('fc2').output)


  for layer in base_model.layers[:-9]:
      layer.trainable = False


  x = base_model.output
  x = Flatten()(x)

  x = Dense(4096, activation='relu', name='fc1')(x)
  x = Dense(4096, activation='relu', name='fc2')(x)

  # and a logistic layer -- let's say we have 200 classes
  predictions = Dense(n_classes, activation='softmax', name='predictions')(x)

  # this is the model we will train
  model = Model(inputs=base_model.input, outputs=predictions)
  
#   for layer in model.layers:
#     print(layer.name, layer.trainable)



  return model 

**Xception**

In [0]:

from keras.preprocessing.image import ImageDataGenerator
import numpy as np

def get_Xception_data_generator(file_num, file_name):
  from keras.applications.xception import preprocess_input
  
  
  print('Xception DATA GENERATOR')
    
  import os
  import zipfile
#   local_zip = 'gdrive/My Drive/Colab Notebooks/UC_Merced/' + file_name
#   local_zip = 'gdrive/My Drive/Colab Notebooks/Sint_Maarten/shape/' + file_name
  local_zip = 'gdrive/My Drive/Colab Notebooks/Sint_Maarten/material/' + file_name


  print(local_zip)
  zip_ref = zipfile.ZipFile(local_zip, 'r')
  zip_ref.extractall('/tmp')
  zip_ref.close()

  base_dir = '/tmp/' + file_name.split(".")[0]
  # base_dir = '/tmp/UC_Merced_filtered_45'

  print(base_dir)

  train_dir = os.path.join(base_dir, 'train')
  validation_dir = os.path.join(base_dir, 'validation')
  test_dir = os.path.join(base_dir, 'test')


  # Adding rescale, rotation_range, width_shift_range, height_shift_range,
  # shear_range, zoom_range, and horizontal flip to our ImageDataGenerator
  train_datagen = ImageDataGenerator(
      preprocessing_function=preprocess_input,
      rotation_range=180,
      horizontal_flip=True,
      vertical_flip=True)

  # Note that the validation and test data should not be augmented!
  val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
  test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

  # Flow training images in batches of 16 using train_datagen generator
  train_generator = train_datagen.flow_from_directory(
          train_dir,  # This is the source directory for training images
          target_size=(299, 299),  # All images will be resized to 150x150
          batch_size=16,
          # Since we use binary_crossentropy loss, we need binary labels
          class_mode='sparse', 
          shuffle=True)

  # Flow validation images in batches of 16 using test_datagen generator
  validation_generator = val_datagen.flow_from_directory(
          validation_dir,
          target_size=(299, 299),
          batch_size=16,
          class_mode='sparse')

  # Flow test images in batches of 16 using test_datagen generator
  test_generator = test_datagen.flow_from_directory(
          test_dir,
          target_size=(299, 299),
          batch_size=100,
          class_mode='sparse')
  
  return train_generator, validation_generator, test_generator




def get_Xception(n_classes):
  
  print('Xception MODEL')
  from keras.applications.xception import Xception

  from keras.models import Model
  from keras.layers import Dense, GlobalAveragePooling2D, Input, Flatten
  

  # create the base pre-trained model
  base_model = Xception(weights='imagenet', include_top=False, input_shape=(299,299, 3))

  for layer in base_model.layers[:95]:
    layer.trainable = False

    # add a global spatial average pooling layer
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  predictions = Dense(n_classes, activation='softmax')(x)


  # this is the model we will train
  model = Model(inputs=base_model.input, outputs=predictions)


  return model 



**Inception**

In [0]:
from keras.preprocessing.image import ImageDataGenerator
import numpy as np


def get_Inception_data_generator(file_num, file_name):
  from keras.applications.inception_v3 import preprocess_input

  
  print('INCEPTION DATA GENERATOR')
    
  import os
  import zipfile
  
#   local_zip = 'gdrive/My Drive/Colab Notebooks/UC_Merced/' + file_name
#   local_zip = 'gdrive/My Drive/Colab Notebooks/Sint_Maarten/shape/' + file_name
  local_zip = 'gdrive/My Drive/Colab Notebooks/Sint_Maarten/material/' + file_name


  print(local_zip)
  zip_ref = zipfile.ZipFile(local_zip, 'r')
  zip_ref.extractall('/tmp')
  zip_ref.close()

  base_dir = '/tmp/' + file_name.split(".")[0]
  # base_dir = '/tmp/UC_Merced_filtered_45'

  print(base_dir)

  train_dir = os.path.join(base_dir, 'train')
  validation_dir = os.path.join(base_dir, 'validation')
  test_dir = os.path.join(base_dir, 'test')


  # Adding rescale, rotation_range, width_shift_range, height_shift_range,
  # shear_range, zoom_range, and horizontal flip to our ImageDataGenerator
  train_datagen = ImageDataGenerator(
      preprocessing_function=preprocess_input,
      rotation_range=180,
      horizontal_flip=True,
      vertical_flip=True)

  # Note that the validation and test data should not be augmented!
  val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
  test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

  # Flow training images in batches of 16 using train_datagen generator
  train_generator = train_datagen.flow_from_directory(
          train_dir,  # This is the source directory for training images
          target_size=(299, 299),  # All images will be resized to 150x150
          batch_size=32,
          # Since we use binary_crossentropy loss, we need binary labels
          class_mode='sparse', 
          shuffle=True)

  # Flow validation images in batches of 16 using test_datagen generator
  validation_generator = val_datagen.flow_from_directory(
          validation_dir,
          target_size=(299, 299),
          batch_size=32,
          class_mode='sparse')

  # Flow test images in batches of 16 using test_datagen generator
  test_generator = test_datagen.flow_from_directory(
          test_dir,
          target_size=(299, 299),
          batch_size=100,
          class_mode='sparse')
  
  return train_generator, validation_generator, test_generator




def get_Inception(n_classes):
  
  print('INCEPTION MODEL')
  
  from keras.applications.vgg16 import  VGG16
  from keras.applications.inception_v3 import InceptionV3
  from keras.models import Model
  from keras.layers import Dense, GlobalAveragePooling2D, Input, Flatten
  


  # create the base pre-trained model
  base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299,299, 3))
  
  for layer in base_model.layers[:249]:
    layer.trainable = False

  # add a global spatial average pooling layer
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  # let's add a fully-connected layer
  x = Dense(1024, activation='relu')(x)
  # and a logistic layer -- let's say we have 200 classes
  predictions = Dense(n_classes, activation='softmax')(x)

  # this is the model we will train
  model = Model(inputs=base_model.input, outputs=predictions)


  return model 


**FUNCTION**

In [0]:

def set_data_generators(file_num, network, file_name, fraction):
  
  if network == 'VGG':
    train_generator, validation_generator, test_generator = get_VGG_data_generator(file_num, file_name, fraction)
    
  elif network == 'Inception':
    train_generator, validation_generator, test_generator = get_Inception_data_generator(file_num, file_name)
    
  elif network == 'Xception':
    train_generator, validation_generator, test_generator = get_Xception_data_generator(file_num, file_name)
  
    
  return train_generator, validation_generator, test_generator



def get_model(network, n_classes):
  
  if network == 'VGG':
    model = get_VGG(n_classes)
          
  elif network == 'Inception':
    model = get_Inception(n_classes)
    
  elif network == 'Xception':
    model = get_Xception(n_classes)
    
  return model
    
              

**Loop**

In [0]:
def loop(file_num, network, fraction):

  #Filename
#   file_name = 'UC_Merced_filtered_'+ str(file_num) +'.zip'
#   file_name = 'roof_shape_'+ str(file_num*10) +'_0_9' + '.zip'
  file_name = 'roof_material_'+ str(file_num*10) +'_0_9' + '.zip'


  #generators
  train_generator, validation_generator, test_generator = set_data_generators(file_num, network, file_name, fraction)

  model = get_model(network,  train_generator.num_classes)
  
  #Class weights
  class_weights = class_weight.compute_class_weight('balanced',
                                                   np.unique(train_generator.classes),
                                                   train_generator.classes)
  
#   print(class_weights)

  #file paths
  PATH = '/content/gdrive/My Drive/Colab Notebooks/Sint_Maarten/pickles/models/' + network + '/' + str(fraction) + '/'
  model_file_path = PATH + 'trained_models/' + str(train_generator.num_classes) + '_' + file_name.split(".")[0]
  csv_file_path = PATH + 'logs/' + str(train_generator.num_classes) + '_' + file_name.split(".")[0] + '.csv'
  
  
  #Callbacks
  callbacks = list()
  callbacks.append(CSVLogger(csv_file_path, separator=',', append=False))
#   callbacks.append(ModelCheckpoint(model_file_path, monitor='val_acc', verbose=1, save_best_only=True,
#                                  save_weights_only=False, mode='auto', period=3))
  
  if network == 'VGG':
    opt = SGD(lr=0.001)
    epoch = 15
  elif network == 'Inception':
    opt = SGD(lr=0.01) 
    epoch = 15
  elif network == 'Xception':
    opt = SGD(lr=0.01)
    epoch = 30
  
  model.compile(loss='sparse_categorical_crossentropy',
                optimizer=opt,
                metrics=['acc'])

  # Fit Model
  model_info = model.fit_generator(train_generator,
                                   epochs=epoch,
                                   steps_per_epoch=train_generator.samples/train_generator.batch_size,
                                   verbose=1,
                                   validation_data=validation_generator,
                                   validation_steps=validation_generator.samples/validation_generator.batch_size,
                                   callbacks=callbacks,
                                   class_weight=class_weights
                                   )
  
  
  #Callbacks
  callbacks = list()
  callbacks.append(CSVLogger(csv_file_path, separator=',', append=True))
  callbacks.append(ModelCheckpoint(model_file_path, monitor='val_acc', verbose=1, save_best_only=True,
                                 save_weights_only=False, mode='auto', period=1))
    
    
  #optimizers
  
  if network == 'VGG':
    opt = SGD(lr=0.001)
    epoch = 45
  elif network == 'Inception':
    opt = SGD(lr=0.001)  
    epoch = 45
  elif network == 'Xception':
    opt = SGD(lr=0.001)
    epoch = 70

   
  model.compile(loss='sparse_categorical_crossentropy',
                optimizer=opt,
                metrics=['acc'])

  # Fit Model
  model_info = model.fit_generator(train_generator,
                                   epochs=epoch,
                                   steps_per_epoch=train_generator.samples/train_generator.batch_size,
                                   verbose=1,
                                   validation_data=validation_generator,
                                   validation_steps=validation_generator.samples/validation_generator.batch_size,
                                   callbacks=callbacks,
                                   class_weight=class_weights
                                   )
  
  
  

  train_generator.reset()
  validation_generator.reset()
  model = []

  K.clear_session()

In [8]:
from google.colab import drive    
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
for network in ['VGG']:
  for fraction in ['10']:

    for file_num in [i*1 for i in range(1,11)]:

      loop(file_num, network, fraction)


VGG DATA GENERATOR
gdrive/My Drive/Colab Notebooks/Sint_Maarten/material/10/roof_material_10_0_9.zip
/tmp/roof_material_10_0_9
Found 814 images belonging to 3 classes.
Found 351 images belonging to 3 classes.
Found 10496 images belonging to 3 classes.
VGG MODEL
Instructions for updating:
Colocations handled automatically by placer.
58892288/58889256 [==============================] - 2s 0us/step
Instructions for updating:
Use tf.cast instead.
Epoch 1/15
51/50 [==============================] - 30s 580ms/step - loss: 1.4849 - acc: 0.5432 - val_loss: 1.0213 - val_acc: 0.5071
Epoch 2/15
51/50 [==============================] - 18s 352ms/step - loss: 0.9656 - acc: 0.5527 - val_loss: 1.0000 - val_acc: 0.5385
Epoch 3/15
51/50 [==============================] - 18s 353ms/step - loss: 0.9175 - acc: 0.5837 - val_loss: 0.9551 - val_acc: 0.5869
Epoch 4/15
12/50 [======>.......................] - ETA: 11s - loss: 0.8620 - acc: 0.6562